### Model implementation

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import scipy.stats as st

Import a test dataset

In [ ]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

Split Dataset

In [ ]:
# 80/20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Choquistic Regression

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class GameBasedChoquisticRegression_2_additive(BaseEstimator, ClassifierMixin):
    def __init__(self, gamma=1.0, max_iter=100, tol=1e-4, random_state=None):
        """
        Game-theoretic logistic regression with a 2-additive Choquet integral aggregation.
        
        Parameters
        ----------
        gamma : float, default=1.0
            Scaling factor inside the sigmoid.
        max_iter : int, default=100
            Maximum number of iterations for the optimizer.
        tol : float, default=1e-4
            Tolerance for termination.
        random_state : int or None, default=None
            Random seed for parameter initialization.
        """
        self.gamma = gamma
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state

    def _init_parameters(self, m):
        rng = np.random.RandomState(self.random_state)
        # Initialize main effects (Shapley values)
        self.phi_ = rng.normal(scale=0.1, size=m)
        # Initialize interaction parameters (Shapley interaction indices)
        self.I_ = rng.normal(scale=0.05, size=(m, m))
        # Force the diagonal to zero (no self-interaction)
        np.fill_diagonal(self.I_, 0)
        # Initialize the intercept
        self.beta0_ = rng.normal()

    def _compute_f_CI(self, X):
        """
        Compute the aggregated value using the 2-additive Choquet integral:
        
        f_CI(v, x_i) = ∑_{j=1}^m x_{i,j} (φ_j - ½∑_{j'≠j} I_{j,j'})
                        + ∑_{j≠j'} (min(x_{i,j}, x_{i,j'}) * I_{j,j'})
        """
        # Main effects: note that I_.sum(axis=1) sums over j'=1..m;
        # since we enforce I[j,j] = 0, this equals the sum over j'≠j.
        main_effects = X @ (self.phi_ - 0.5 * self.I_.sum(axis=1))
        # Interaction effects: use broadcasting to compute pairwise minima.
        interactions = np.sum(np.minimum(X[:, :, None], X[:, None, :]) * self.I_, axis=(1, 2))
        return main_effects + interactions

    def _sigmoid(self, f):
        return 1 / (1 + np.exp(-self.gamma * (f - self.beta0_)))

    def _negative_log_likelihood(self, params, X, y):
        m = X.shape[1]
        # Unpack the parameter vector.
        self.phi_ = params[:m]
        self.I_ = params[m:-1].reshape((m, m))
        self.beta0_ = params[-1]
        # Compute the aggregated value and then the predicted probability.
        f = self._compute_f_CI(X)
        proba = self._sigmoid(f)
        # Clip probabilities for numerical stability.
        proba = np.clip(proba, 1e-15, 1 - 1e-15)
        # Return the mean logistic loss.
        return -np.mean(y * np.log(proba) + (1 - y) * np.log(1 - proba))

    def fit(self, X, y):
        """
        Fit the model by minimizing the logistic loss with the Choquet integral aggregation.
        """
        X, y = check_X_y(X, y)
        m = X.shape[1]
        self._init_parameters(m)
        
        # Pack parameters into a single vector.
        initial_params = np.concatenate([self.phi_, self.I_.flatten(), [self.beta0_]])
        n_params = len(initial_params)
        
        # Set bounds to enforce I[j,j] = 0 for all j.
        bounds = [(None, None)] * n_params
        for j in range(m):
            diag_index = m + j * m + j  # Index for I[j,j] in the flattened I.
            bounds[diag_index] = (0, 0)
        
        res = minimize(self._negative_log_likelihood, initial_params, args=(X, y),
                       method='L-BFGS-B', bounds=bounds,
                       options={'maxiter': self.max_iter, 'gtol': self.tol})
        
        # Optionally, you might check res.success here.
        self.phi_ = res.x[:m]
        self.I_ = res.x[m:-1].reshape((m, m))
        self.beta0_ = res.x[-1]
        
        return self

    def predict_proba(self, X):
        check_is_fitted(self)
        X = check_array(X)
        f = self._compute_f_CI(X)
        proba = self._sigmoid(f)
        return np.vstack([1 - proba, proba]).T

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)


## ML Regression

In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class GameMultilinearRegression(BaseEstimator, ClassifierMixin):
    def __init__(self, gamma=1.0, max_iter=100, tol=1e-4, random_state=None):
        self.gamma = gamma
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        
    def _init_parameters(self, m):
        rng = np.random.RandomState(self.random_state)
        self.phi_ = rng.normal(scale=0.1, size=m)
        self.I_ = rng.normal(scale=0.05, size=(m, m))
        # Enforce zero diagonal at initialization:
        np.fill_diagonal(self.I_, 0)
        self.beta0_ = rng.normal()
        
    def _compute_f_ML(self, X):
        # Compute main effects and interaction effects as in the 2-additive model
        main_effects = X @ (self.phi_ - 0.5 * self.I_.sum(axis=1))
        interactions = np.sum(X[:, :, None] * X[:, None, :] * self.I_, axis=(1,2))
        # Subtract half the diagonal to cancel any unintended diagonal contribution
        diag_correction = np.diag(X @ self.I_ @ X.T) / 2
        return main_effects + interactions - diag_correction

    def _sigmoid(self, f):
        return 1 / (1 + np.exp(-self.gamma * (f - self.beta0_)))
    
    def _negative_log_likelihood(self, params, X, y):
        m = X.shape[1]
        # Unpack parameters
        self.phi_ = params[:m]
        self.I_ = params[m:-1].reshape((m, m))
        self.beta0_ = params[-1]
        
        # Compute the aggregated value
        f = self._compute_f_ML(X)
        proba = self._sigmoid(f)
        proba = np.clip(proba, 1e-15, 1-1e-15)
        # Return the mean logistic loss
        return -np.mean(y * np.log(proba) + (1 - y) * np.log(1 - proba))
    
    def fit(self, X, y):
        X, y = check_X_y(X, y)
        m = X.shape[1]
        self._init_parameters(m)
        
        initial_params = np.concatenate([
            self.phi_,
            self.I_.flatten(),
            [self.beta0_]
        ])
        
        # Create bounds to force zero on the diagonal of I.
        n_params = len(initial_params)
        bounds = [(None, None)] * n_params
        # Identify indices corresponding to the diagonal of I
        # phi occupies indices [0, m)
        # I occupies indices [m, m+m*m)
        for j in range(m):
            diag_index = m + j * m + j  # for row j, col j in I.flatten()
            bounds[diag_index] = (0, 0)  # force I[j,j] = 0
        
        res = minimize(self._negative_log_likelihood, initial_params,
                       args=(X, y), method='L-BFGS-B',
                       bounds=bounds,
                       options={'maxiter': self.max_iter, 'gtol': self.tol})
        
        # Optionally, check res.success and raise a warning if needed.
        
        # Update parameters with optimized values
        self.phi_ = res.x[:m]
        self.I_ = res.x[m:-1].reshape((m, m))
        self.beta0_ = res.x[-1]
        
        return self

    def predict_proba(self, X):
        check_is_fitted(self)
        X = check_array(X)
        f = self._compute_f_ML(X)
        proba = self._sigmoid(f)
        return np.vstack([1 - proba, proba]).T

    def predict(self, X):
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)


In [ ]:
# GPT v2
import numpy as np
from scipy.optimize import minimize
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_array, check_is_fitted

class GameMultilinearRegression_2_additive(BaseEstimator, ClassifierMixin):
    """
    2-Additive Game-Theoretic Logistic Regression using a multilinear aggregation function.
    
    The model predicts the probability via
        p(y=1 | x) = σ(γ ( f(x) - β₀ ) )
    where
        f(x) = ∑₍j₌1₎ᵐ x_j ( φ_j - 0.5 * ∑ₖ₍≠j₎ I_{j,k} ) + xᵀ I x,
    and I is a symmetric matrix with zeros on the diagonal.
    
    Parameters
    ----------
    gamma : float, default=1.0
        Scaling hyperparameter inside the logistic function.
        
    max_iter : int, default=100
        Maximum number of iterations for the optimizer.
        
    tol : float, default=1e-5
        Tolerance for termination.
        
    verbose : bool, default=False
        If True, prints convergence messages.
    """
    
    def __init__(self, gamma=1.0, max_iter=100, tol=1e-5, verbose=False):
        self.gamma = gamma
        self.max_iter = max_iter
        self.tol = tol
        self.verbose = verbose

    def _unpack_params(self, theta, m):
        """
        Given parameter vector theta of length 1 + m + m*(m-1)//2, extract:
          - beta0 (scalar)
          - phi (length m)
          - I (m x m symmetric matrix with zeros on the diagonal)
        """
        beta0 = theta[0]
        phi = theta[1:1+m]
        I_vec = theta[1+m:]
        I = np.zeros((m, m))
        idx = 0
        for j in range(m):
            for k in range(j+1, m):
                I[j, k] = I_vec[idx]
                I[k, j] = I_vec[idx]
                idx += 1
        return beta0, phi, I

    def _objective_grad(self, theta, X, y):
        """
        Compute the objective (binary cross-entropy loss) and its gradient.
        
        Parameters
        ----------
        theta : ndarray, shape (p,)
            Current parameter vector.
        X : ndarray, shape (n_samples, m)
            Input features.
        y : ndarray, shape (n_samples,)
            Binary targets (0 or 1).
        
        Returns
        -------
        loss : float
            The value of the loss.
        grad : ndarray, shape (p,)
            The gradient with respect to theta.
        """
        n, m = X.shape
        beta0, phi, I = self._unpack_params(theta, m)
        
        # Compute row sum of I (recall: I[j,j]=0)
        r = np.sum(I, axis=1)  # shape (m,)
        
        # First term: ∑₍j₎ x_j ( φ_j - 0.5 * r_j )
        A = phi - 0.5 * r  # shape (m,)
        term1 = X.dot(A)   # shape (n,)
        
        # Second term: xᵀ I x for each sample.
        term2 = np.sum(X * (X.dot(I)), axis=1)  # shape (n,)
        
        # Aggregated value f(x)
        f = term1 + term2
        
        # Logits: gamma*(f - beta0)
        logits = self.gamma * (f - beta0)
        sigma = 1.0 / (1.0 + np.exp(-logits))
        
        # Binary cross-entropy loss (with small epsilon for numerical stability)
        eps = 1e-15
        loss = -np.sum(y * np.log(sigma + eps) + (1 - y) * np.log(1 - sigma + eps))
        
        # Compute delta = dL/d(logit) = sigma - y
        delta = sigma - y  # shape (n,)
        
        # Gradient with respect to beta0:
        # d(logit)/d(beta0) = -gamma, so:
        grad_beta0 = -self.gamma * np.sum(delta)
        
        # Gradient with respect to phi:
        # d(f)/d(phi_j) = x_j  =>  grad_phi[j] = gamma * sum_i (delta_i * x_{ij})
        grad_phi = self.gamma * (X.T.dot(delta))  # shape (m,)
        
        # Gradient with respect to I_{j,k} for j < k:
        # For a given sample i, 
        #   d(f)/dI_{j,k} = -0.5*(x_{i,j} + x_{i,k})  [from term1] + 2*x_{i,j}*x_{i,k} [from term2].
        grad_I_list = []
        for j in range(m):
            for k in range(j+1, m):
                deriv = 2 * X[:, j] * X[:, k] - 0.5 * (X[:, j] + X[:, k])
                grad_I_jk = self.gamma * np.sum(delta * deriv)
                grad_I_list.append(grad_I_jk)
        grad_I = np.array(grad_I_list)  # length m*(m-1)/2
        
        # Combine all gradients in the same order as theta.
        grad = np.concatenate(([grad_beta0], grad_phi, grad_I))
        return loss, grad

    def fit(self, X, y):
        """
        Fit the model.
        
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Feature matrix (each attribute should be in [0,1]).
        y : array-like of shape (n_samples,)
            Binary targets.
            
        Returns
        -------
        self : object
            Fitted estimator.
        """
        X = check_array(X)
        y = np.asarray(y).flatten()
        n, m = X.shape
        self.n_features_in_ = m  # store number of features
        
        # Total number of parameters: 1 (beta0) + m (phi) + m*(m-1)/2 (for I)
        num_params = 1 + m + (m * (m - 1)) // 2
        theta0 = np.zeros(num_params)  # initialize all parameters to zero
        
        # Optimize the objective using L-BFGS-B
        result = minimize(
            fun=self._objective_grad,
            x0=theta0,
            args=(X, y),
            method='L-BFGS-B',
            jac=True,
            options={'maxiter': self.max_iter, 'ftol': self.tol, 'disp': self.verbose}
        )
        self.theta_ = result.x
        self.n_iter_ = result.nit
        self.loss_ = result.fun
        
        # Unpack fitted parameters
        self.beta0_, self.phi_, self.I_ = self._unpack_params(self.theta_, m)
        return self

    def decision_function(self, X):
        """
        Compute the decision function (logits) for samples in X.
        
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        
        Returns
        -------
        logits : ndarray of shape (n_samples,)
        """
        X = check_array(X)
        n, m = X.shape
        check_is_fitted(self, 'theta_')
        r = np.sum(self.I_, axis=1)  # row sums of I
        A = self.phi_ - 0.5 * r
        term1 = X.dot(A)
        term2 = np.sum(X * (X.dot(self.I_)), axis=1)
        f = term1 + term2
        logits = self.gamma * (f - self.beta0_)
        return logits

    def predict_proba(self, X):
        """
        Compute probabilities for each class.
        
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        
        Returns
        -------
        proba : ndarray of shape (n_samples, 2)
            The probability of the negative class is in column 0 and
            the probability of the positive class is in column 1.
        """
        logits = self.decision_function(X)
        proba_1 = 1.0 / (1.0 + np.exp(-logits))
        proba_0 = 1 - proba_1
        return np.vstack([proba_0, proba_1]).T

    def predict(self, X):
        """
        Predict class labels for samples in X.
        
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
        
        Returns
        -------
        labels : ndarray of shape (n_samples,)
            Predicted binary class labels.
        """
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)


In [ ]:
# GPT v3
import numpy as np
from scipy.optimize import minimize
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_array, check_X_y, check_is_fitted

class GameTheoreticMultilinearLogisticRegression_2_additive(BaseEstimator, ClassifierMixin):
    """
    2-Additive Game-Theoretic Multilinear Logistic Regression.
    
    This estimator models the aggregated value as:
    
        f(x) = sum_{j=1}^m x_j (phi_j - 0.5 * sum_{k != j} I_{j,k})
               + sum_{j != k} x_j x_k I_{j,k},
    
    where I is a symmetric matrix with zero diagonal (i.e. I_{j,j}=0). The prediction is then
    given by:
    
        p(y=1 | x) = σ(γ ( f(x) - β₀ )),
    
    with γ a scaling hyperparameter and σ the logistic sigmoid.
    
    Parameters
    ----------
    gamma : float, default=1.0
        Scaling parameter in the logistic function.
    max_iter : int, default=100
        Maximum number of iterations for the optimizer.
    tol : float, default=1e-4
        Tolerance for termination of optimization.
    random_state : int or None, default=None
        Seed for random number generation.
    """
    
    def __init__(self, gamma=1.0, max_iter=100, tol=1e-4, random_state=None):
        self.gamma = gamma
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        
    def _init_parameters(self, m):
        """Initialize parameters: phi (length m), I_upper (m*(m-1)/2 independent off-diagonals), and beta0."""
        rng = np.random.RandomState(self.random_state)
        self.phi_ = rng.normal(scale=0.1, size=m)
        # For a symmetric m x m matrix with zero diagonal, we need only m*(m-1)/2 parameters.
        self.I_upper_ = rng.normal(scale=0.05, size=(m * (m - 1)) // 2)
        self.beta0_ = rng.normal()
        
    def _pack_params(self):
        """Pack parameters into a single vector: [beta0, phi (m,), I_upper (m*(m-1)/2,)]."""
        return np.concatenate(([self.beta0_], self.phi_, self.I_upper_))
    
    def _unpack_params(self, theta, m):
        """
        Unpack theta into beta0, phi, and reconstruct the full symmetric interaction matrix I
        (with zeros on the diagonal) from the upper-triangular entries.
        """
        beta0 = theta[0]
        phi = theta[1:1+m]
        I_upper = theta[1+m:]
        I = np.zeros((m, m))
        idx = 0
        for j in range(m):
            for k in range(j+1, m):
                I[j, k] = I_upper[idx]
                I[k, j] = I_upper[idx]
                idx += 1
        return beta0, phi, I
    
    def _compute_f(self, X, phi, I):
        """
        Compute the aggregated value f(x) for each sample.
        
        f(x) = X · (phi - 0.5 * row_sum(I)) + sum_{j,k} x_j x_k I_{j,k}.
        """
        # row_sum: for each feature j, sum over k (note I[j,j]=0 so this is sum_{k != j})
        row_sum = np.sum(I, axis=1)
        main_effects = X.dot(phi - 0.5 * row_sum)
        # Interaction effects: compute x^T I x for each sample.
        interactions = np.sum(X * (X.dot(I)), axis=1)
        return main_effects + interactions
    
    def _loss_and_grad(self, theta, X, y):
        """
        Compute the mean negative log-likelihood and its gradient with respect to theta.
        
        Parameters
        ----------
        theta : 1D ndarray
            Current parameter vector.
        X : ndarray, shape (n_samples, m)
            Input features.
        y : ndarray, shape (n_samples,)
            Binary targets.
            
        Returns
        -------
        loss : float
            The mean negative log-likelihood.
        grad : 1D ndarray
            The gradient with respect to theta.
        """
        n, m = X.shape
        beta0, phi, I = self._unpack_params(theta, m)
        # Compute the aggregated function f(x)
        f = self._compute_f(X, phi, I)
        # Compute logits: note the model is defined as logit = gamma*(f(x) - beta0)
        logits = self.gamma * (f - beta0)
        sigma = 1.0 / (1.0 + np.exp(-logits))
        # Mean negative log-likelihood (with a small epsilon for numerical stability)
        eps = 1e-15
        loss = - np.mean(y * np.log(sigma + eps) + (1 - y) * np.log(1 - sigma + eps))
        
        # Compute delta = sigma - y (derivative of loss with respect to logits)
        delta = sigma - y  # shape (n,)
        
        # Gradients (remember: loss is the mean over samples)
        # Gradient with respect to beta0:
        grad_beta0 = - self.gamma * np.mean(delta)
        
        # Gradient with respect to phi_j:
        grad_phi = self.gamma * (X.T.dot(delta)) / n
        
        # Gradient with respect to each independent parameter I_{j,k} for j < k.
        grad_I = []
        # For each pair (j, k), the derivative of f(x) with respect to I_{j,k} is:
        # d f/d I_{j,k} = 2*x_j*x_k - 0.5*(x_j + x_k)
        for j in range(m):
            for k in range(j+1, m):
                deriv = 2 * X[:, j] * X[:, k] - 0.5 * (X[:, j] + X[:, k])
                grad_I_jk = self.gamma * np.mean(delta * deriv)
                grad_I.append(grad_I_jk)
        grad_I = np.array(grad_I)
        
        # Pack gradients in the same order as theta.
        grad = np.concatenate(([grad_beta0], grad_phi, grad_I))
        return loss, grad

    def fit(self, X, y):
        """
        Fit the game-theoretic multilinear logistic regression model.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Input features. Each feature value should be in [0,1].
        y : array-like, shape (n_samples,)
            Binary targets (0 or 1).
        
        Returns
        -------
        self : object
            Fitted estimator.
        """
        X, y = check_X_y(X, y)
        n, m = X.shape
        self._init_parameters(m)
        initial_theta = self._pack_params()
        
        res = minimize(fun=self._loss_and_grad, x0=initial_theta, args=(X, y),
                       method='L-BFGS-B', jac=True,
                       options={'maxiter': self.max_iter, 'ftol': self.tol})
        
        if not res.success:
            print("Optimization did not converge:", res.message)
        
        self.theta_ = res.x
        self.beta0_, self.phi_, self.I_ = self._unpack_params(self.theta_, m)
        self.n_iter_ = res.nit
        self.loss_ = res.fun
        self.is_fitted_ = True
        return self
    
    def decision_function(self, X):
        """
        Compute the decision function (logits) for samples in X.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
        
        Returns
        -------
        logits : ndarray, shape (n_samples,)
        """
        X = check_array(X)
        check_is_fitted(self, 'is_fitted_')
        n, m = X.shape
        f = self._compute_f(X, self.phi_, self.I_)
        logits = self.gamma * (f - self.beta0_)
        return logits
    
    def predict_proba(self, X):
        """
        Compute class probabilities for samples in X.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
        
        Returns
        -------
        proba : ndarray, shape (n_samples, 2)
            The first column is P(y=0) and the second column is P(y=1).
        """
        logits = self.decision_function(X)
        sigma = 1.0 / (1.0 + np.exp(-logits))
        return np.vstack([1 - sigma, sigma]).T
    
    def predict(self, X):
        """
        Predict binary class labels for samples in X.
        
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
        
        Returns
        -------
        labels : ndarray, shape (n_samples,)
            Predicted binary labels (0 or 1).
        """
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.5).astype(int)
